In [190]:
#from master import MASTERModel
import pickle
import numpy as np
import time
import pandas as pd
import sys, os
import qlib
from qlib.data.dataset import TSDataSampler

# Move up one directory from MASTER-master
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)


from alpha_101.alpha_generator import get_alpha101_table_from_db
from alpha_101.alpha_generator import generate_alphas

In [164]:
# Call generate_alphas() which returns (df, final_df)
df, final_df = generate_alphas(input_schema = 'datacollection',
                    input_table_name = 'stock_data',
                    save = True, 
                    output_schema = 'datacollection',
                    output_table_name = 'alpha101',
                    if_return = True)


Running function alpha001 for ADBE...
Running function alpha002 for ADBE...
Running function alpha003 for ADBE...
Running function alpha004 for ADBE...
Running function alpha006 for ADBE...
Running function alpha007 for ADBE...
Running function alpha008 for ADBE...
Running function alpha009 for ADBE...
Running function alpha010 for ADBE...
Running function alpha011 for ADBE...
Running function alpha012 for ADBE...
Running function alpha013 for ADBE...
Running function alpha014 for ADBE...
Running function alpha015 for ADBE...
Running function alpha016 for ADBE...
Running function alpha017 for ADBE...
Running function alpha018 for ADBE...
Running function alpha019 for ADBE...
Running function alpha020 for ADBE...
Running function alpha021 for ADBE...
Running function alpha022 for ADBE...
Running function alpha023 for ADBE...
Running function alpha024 for ADBE...
Running function alpha025 for ADBE...
Running function alpha026 for ADBE...
Running function alpha028 for ADBE...
Running func

/Users/lilingyan/QF5214_2025_G8/TeamThree/qlib_env/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Running function alpha028 for BIIB...
Running function alpha029 for BIIB...
Running function alpha030 for BIIB...
Running function alpha031 for BIIB...
Running function alpha032 for BIIB...
Running function alpha033 for BIIB...
Running function alpha034 for BIIB...
Running function alpha035 for BIIB...
Running function alpha036 for BIIB...
Running function alpha037 for BIIB...
Running function alpha038 for BIIB...
Running function alpha039 for BIIB...
Running function alpha040 for BIIB...
Running function alpha041 for BIIB...
Running function alpha042 for BIIB...
Running function alpha043 for BIIB...
Running function alpha044 for BIIB...
Running function alpha045 for BIIB...
Running function alpha046 for BIIB...
Running function alpha047 for BIIB...
Running function alpha048 for BIIB...
Running function alpha049 for BIIB...
Running function alpha050 for BIIB...
Running function alpha051 for BIIB...
Running function alpha052 for BIIB...
Running function alpha053 for BIIB...
Running func

In [166]:
df['Ticker'].unique()

<StringArray>
[ 'ADBE',   'AMD',  'ABNB', 'GOOGL',  'GOOG',  'AMZN',   'AEP',  'AMGN',
   'ADI',  'ANSS',
 ...
  'TSLA',   'TXN',   'TTD',  'VRSK',  'VRTX',   'WBD',  'WDAY',   'XEL',
    'ZS',  'CTAS']
Length: 101, dtype: string

In [167]:
company_list = ["ADBE", "AMD", "ABNB", "GOOGL", "GOOG", "AMZN", "AEP", "AMGN", 
"ADI", "ANSS", "AAPL", "AMAT", "APP", "ARM", "ASML", "AZN", 
"TEAM", "ADSK", "ADP", "AXON", "BKR", "BIIB", "BKNG", "AVGO", 
"CDNS", "CDW", "CHTR", "CTAS", "CSCO", "CCEP", "CTSH", "CMCSA", 
"CEG", "CPRT", "CSGP", "COST", "CRWD", "CSX", "DDOG", "DXCM","VRTX", "WBD", "WDAY", "XEL", "ZS","QCOM", "REGN", "ROP", "ROST" ]    

In [168]:
df_all, final_df_all, df_index = get_alpha101_table_from_db()

In [170]:
# 1) Drop the "IndexTicker" column if it exists
df_index = df_index.drop(columns="IndexTicker", errors="ignore")

# 2) Pivot the DataFrame
df_index_pivot = df_index.pivot(
    index="Date",                # Each Date becomes a row
    columns="IndexName",         # Each unique IndexName becomes columns
    values=["Open", "High", "Low", "Close", "Adj_Close", "Volume"]
)

# 3) Flatten and rename columns to single-level format "<IndexName>_<Column>"
df_index_pivot.columns = [
    f"{col[1]}_{col[0]}"  # col is a tuple like ("Open", "S&P500")
    for col in df_index_pivot.columns
]

In [171]:
df_index_pivot = df_index_pivot.drop(columns=["Russell 1000_Volume", 
                                              "Russell 3000_Volume", 
                                              "Wilshire 5000_Volume"])


# 4) Move the 'Date' index back into a column
df_index_pivot.reset_index(inplace=True)

# Now df_index_pivot has columns like:
# ['Date', 'S&P500_Open', 'S&P500_High', 'S&P500_Low', 'S&P500_Close', 'S&P500_Adj_Close', 'S&P500_Volume', ...]

df_index_pivot

,Date,NASDAQ 100_Open,Russell 1000_Open,Russell 3000_Open,S&P 500_Open,Wilshire 5000_Open,NASDAQ 100_High,Russell 1000_High,Russell 3000_High,S&P 500_High,...,Russell 3000_Close,S&P 500_Close,Wilshire 5000_Close,NASDAQ 100_Adj_Close,Russell 1000_Adj_Close,Russell 3000_Adj_Close,S&P 500_Adj_Close,Wilshire 5000_Adj_Close,NASDAQ 100_Volume,S&P 500_Volume
0,2022-01-03,16395.509766,2645.679932,2788.020020,4778.140137,48461.160156,16504.140625,2660.949951,2804.939941,4796.640137,...,2804.929932,4796.560059,48789.359375,16501.769531,2660.780029,2804.929932,4796.560059,48789.359375,4.429960e+09,3.831020e+09
1,2022-01-04,16513.869141,2660.989990,2788.020020,4804.509766,48789.359375,16513.869141,2672.000000,2817.110107,4818.620117,...,2800.409912,4793.540039,48660.750000,16279.730469,2656.489990,2800.409912,4793.540039,48660.750000,5.131110e+09,4.683170e+09
2,2022-01-05,16190.549805,2655.800049,2799.729980,4787.990234,48660.750000,16249.230469,2657.149902,2801.260010,4797.700195,...,2739.340088,4700.580078,47534.621094,15771.780273,2600.530029,2739.340088,4700.580078,47534.621094,5.031850e+09,4.887960e+09
3,2022-01-06,15675.219727,2600.379883,2739.229980,4693.390137,47534.621094,15900.599609,2616.030029,2756.689941,4725.009766,...,2739.199951,4696.049805,47542.789062,15765.360352,2599.429932,2739.199951,4696.049805,47542.789062,4.790820e+09,4.295280e+09
4,2022-01-07,15774.129883,2599.300049,2739.060059,4697.660156,47542.789062,15835.000000,2607.590088,2748.270020,4707.950195,...,2725.600098,4677.029785,47321.140625,15592.190430,2587.729980,2725.600098,4677.029785,47321.140625,4.238070e+09,4.181510e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
802,2025-03-17,19670.689453,3079.219971,3202.620117,5635.600098,56252.058594,19943.810547,3119.679932,3245.120117,5703.520020,...,3229.610107,5675.120117,56672.839844,19812.240234,3104.459961,3229.610107,5675.120117,56672.839844,6.306340e+09,5.031770e+09
803,2025-03-18,19657.099609,3100.090088,3224.760010,5654.529785,56672.839844,19676.050781,3100.090088,3224.760010,5654.529785,...,3194.760010,5614.660156,56063.410156,19483.359375,3070.689941,3194.760010,5614.660156,56063.410156,6.450530e+09,4.666270e+09
804,2025-03-19,19576.320312,3078.689941,3202.840088,5632.370117,56063.410156,19921.619141,3127.000000,3253.659912,5715.330078,...,3231.800049,5675.290039,56715.878906,19736.660156,3105.699951,3231.800049,5675.290039,56715.878906,6.302310e+09,4.660090e+09
805,2025-03-20,19558.279297,3097.810059,3223.020020,5646.919922,56715.878906,19888.849609,3125.179932,3251.580078,5711.149902,...,3223.570068,5662.890137,56569.429688,19677.609375,3098.360107,3223.570068,5662.890137,56569.429688,6.098330e+09,4.678420e+09


In [172]:
# Merge on "Date" and "Ticker" (adjust join type if needed)
combined_df = pd.merge(df_all, final_df_all, on=["Date", "Ticker"], how="inner")

In [173]:
# 5) (Optional) Merge with your main DataFrame on 'Date'
df_merged = pd.merge(
    combined_df,
    df_index_pivot,
    on='Date',    # or how='left'/'right'/'outer' if needed
    how='left'
)

df_merged

,Date,Open,High,Low,Close,Volume,Market_Cap,Ticker,IndClass_Sector,IndClass_Industry,...,Russell 3000_Close,S&P 500_Close,Wilshire 5000_Close,NASDAQ 100_Adj_Close,Russell 1000_Adj_Close,Russell 3000_Adj_Close,S&P 500_Adj_Close,Wilshire 5000_Adj_Close,NASDAQ 100_Volume,S&P 500_Volume
0,2022-01-03,566.650024,575.000000,553.270020,564.369995,2862700.0,1.918454e+11,ADBE,Technology,Software - Application,...,2804.929932,4796.560059,48789.359375,16501.769531,2660.780029,2804.929932,4796.560059,48789.359375,4.429960e+09,3.831020e+09
1,2022-01-04,563.900024,564.299988,541.309998,554.000000,4344800.0,1.918454e+11,ADBE,Technology,Software - Application,...,2800.409912,4793.540039,48660.750000,16279.730469,2656.489990,2800.409912,4793.540039,48660.750000,5.131110e+09,4.683170e+09
2,2022-01-05,535.000000,538.750000,514.000000,514.429993,7394800.0,1.918454e+11,ADBE,Technology,Software - Application,...,2739.340088,4700.580078,47534.621094,15771.780273,2600.530029,2739.340088,4700.580078,47534.621094,5.031850e+09,4.887960e+09
3,2022-01-06,510.000000,526.309998,508.200012,514.119995,3913600.0,1.918454e+11,ADBE,Technology,Software - Application,...,2739.199951,4696.049805,47542.789062,15765.360352,2599.429932,2739.199951,4696.049805,47542.789062,4.790820e+09,4.295280e+09
4,2022-01-07,513.659973,520.419983,504.000000,510.700012,3605400.0,1.918454e+11,ADBE,Technology,Software - Application,...,2725.600098,4677.029785,47321.140625,15592.190430,2587.729980,2725.600098,4677.029785,47321.140625,4.238070e+09,4.181510e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80034,2025-03-21,505.420013,510.079987,501.070007,503.200012,2280500.0,1.292167e+11,VRTX,Healthcare,Biotechnology,...,3224.929932,5667.560059,56595.609375,19753.970703,3100.550049,3224.929932,5667.560059,56595.609375,8.837930e+09,9.367460e+09
80035,2025-03-21,10.560000,10.790000,10.450000,10.740000,55497700.0,2.636412e+10,WBD,Communication Services,Entertainment,...,3224.929932,5667.560059,56595.609375,19753.970703,3100.550049,3224.929932,5667.560059,56595.609375,8.837930e+09,9.367460e+09
80036,2025-03-21,246.990005,250.190002,245.229996,249.490005,2831800.0,6.636434e+10,WDAY,Technology,Software - Application,...,3224.929932,5667.560059,56595.609375,19753.970703,3100.550049,3224.929932,5667.560059,56595.609375,8.837930e+09,9.367460e+09
80037,2025-03-21,69.809998,70.360001,68.820000,69.110001,6689200.0,3.970736e+10,XEL,Utilities,Utilities - Regulated Electric,...,3224.929932,5667.560059,56595.609375,19753.970703,3100.550049,3224.929932,5667.560059,56595.609375,8.837930e+09,9.367460e+09


In [176]:
filtered_df_merged = df_merged[df_merged['Ticker'].isin(company_list)]

filtered_df_merged

,Date,Open,High,Low,Close,Volume,Market_Cap,Ticker,IndClass_Sector,IndClass_Industry,...,Russell 3000_Close,S&P 500_Close,Wilshire 5000_Close,NASDAQ 100_Adj_Close,Russell 1000_Adj_Close,Russell 3000_Adj_Close,S&P 500_Adj_Close,Wilshire 5000_Adj_Close,NASDAQ 100_Volume,S&P 500_Volume
0,2022-01-03,566.650024,575.000000,553.270020,564.369995,2862700.0,1.918454e+11,ADBE,Technology,Software - Application,...,2804.929932,4796.560059,48789.359375,16501.769531,2660.780029,2804.929932,4796.560059,48789.359375,4.429960e+09,3.831020e+09
1,2022-01-04,563.900024,564.299988,541.309998,554.000000,4344800.0,1.918454e+11,ADBE,Technology,Software - Application,...,2800.409912,4793.540039,48660.750000,16279.730469,2656.489990,2800.409912,4793.540039,48660.750000,5.131110e+09,4.683170e+09
2,2022-01-05,535.000000,538.750000,514.000000,514.429993,7394800.0,1.918454e+11,ADBE,Technology,Software - Application,...,2739.340088,4700.580078,47534.621094,15771.780273,2600.530029,2739.340088,4700.580078,47534.621094,5.031850e+09,4.887960e+09
3,2022-01-06,510.000000,526.309998,508.200012,514.119995,3913600.0,1.918454e+11,ADBE,Technology,Software - Application,...,2739.199951,4696.049805,47542.789062,15765.360352,2599.429932,2739.199951,4696.049805,47542.789062,4.790820e+09,4.295280e+09
4,2022-01-07,513.659973,520.419983,504.000000,510.700012,3605400.0,1.918454e+11,ADBE,Technology,Software - Application,...,2725.600098,4677.029785,47321.140625,15592.190430,2587.729980,2725.600098,4677.029785,47321.140625,4.238070e+09,4.181510e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80034,2025-03-21,505.420013,510.079987,501.070007,503.200012,2280500.0,1.292167e+11,VRTX,Healthcare,Biotechnology,...,3224.929932,5667.560059,56595.609375,19753.970703,3100.550049,3224.929932,5667.560059,56595.609375,8.837930e+09,9.367460e+09
80035,2025-03-21,10.560000,10.790000,10.450000,10.740000,55497700.0,2.636412e+10,WBD,Communication Services,Entertainment,...,3224.929932,5667.560059,56595.609375,19753.970703,3100.550049,3224.929932,5667.560059,56595.609375,8.837930e+09,9.367460e+09
80036,2025-03-21,246.990005,250.190002,245.229996,249.490005,2831800.0,6.636434e+10,WDAY,Technology,Software - Application,...,3224.929932,5667.560059,56595.609375,19753.970703,3100.550049,3224.929932,5667.560059,56595.609375,8.837930e+09,9.367460e+09
80037,2025-03-21,69.809998,70.360001,68.820000,69.110001,6689200.0,3.970736e+10,XEL,Utilities,Utilities - Regulated Electric,...,3224.929932,5667.560059,56595.609375,19753.970703,3100.550049,3224.929932,5667.560059,56595.609375,8.837930e+09,9.367460e+09


In [177]:
filtered_df_merged.fillna(0, inplace=True)

/var/folders/74/m6y7v_2j0p53ss3v0zp9rtbc0000gn/T/ipykernel_95203/3409798551.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_merged.fillna(0, inplace=True)


In [178]:
# Example of dropping non-numeric columns:
filtered_df_merged = filtered_df_merged.drop(columns=["IndClass_Sector", "IndClass_Industry"])

df_sorted = filtered_df_merged.sort_values(by='Date').reset_index(drop=True)

# Compute indices for splits
n = len(df_sorted)
train_end = int(n * 0.6)  # 60%
valid_end = int(n * 0.8)  # 80%

# Slice the DataFrame
df_train = df_sorted.iloc[:train_end]
df_valid = df_sorted.iloc[train_end:valid_end]
df_test = df_sorted.iloc[valid_end:]

df_train.fillna(0, inplace=True)
df_valid.fillna(0, inplace=True)
df_test.fillna(0, inplace=True)

/var/folders/74/m6y7v_2j0p53ss3v0zp9rtbc0000gn/T/ipykernel_95203/2795876467.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train.fillna(0, inplace=True)
/var/folders/74/m6y7v_2j0p53ss3v0zp9rtbc0000gn/T/ipykernel_95203/2795876467.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid.fillna(0, inplace=True)
/var/folders/74/m6y7v_2j0p53ss3v0zp9rtbc0000gn/T/ipykernel_95203/2795876467.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [179]:
def convert_data_qlibformat(df):
    import pandas as pd
    import numpy as np
    from qlib.data.dataset import TSDataSampler  # adjust import as needed

    # Create a copy to avoid SettingWithCopyWarning
    df = df.copy()
    
    # 1) Ensure the "Date" column is a datetime type
    df["Date"] = pd.to_datetime(df["Date"])
    
    # 2) Separate label and features BEFORE setting the index
    #    Retain "Date" and "Ticker" for indexing, but remove them from features.
    df_label = df[["Return"]].copy()
    # Exclude "Date" and "Ticker" along with "Return" from features
    df_feature = df.drop(columns=["Return", "Date", "Ticker"], errors="ignore")
    
    # 3) Normalize feature columns (ensure they are numeric)
    df_feature = df_feature.apply(pd.to_numeric, errors='coerce')
    feature_mean = df_feature.mean()
    feature_std = df_feature.std()
    eps = 1e-8  # to avoid division by zero
    normalized_feature = (df_feature - feature_mean) / (feature_std + eps)
    df_feature = normalized_feature
    
    # 4) Now set the index of the original DataFrame using "Date" and "Ticker"
    #    These columns will be used as the index ("datetime" and "instrument")
    df_index = df.set_index(["Date", "Ticker"])
    
    # 5) Assign the index to both label and feature DataFrames
    df_feature.index = df_index.index
    df_label.index = df_index.index

    # 6) Construct MultiIndex for the columns:
    #    For features, use top-level "feature"; for the label, use "label"
    df_feature.columns = pd.MultiIndex.from_product([["feature"], df_feature.columns])
    df_label.columns = pd.MultiIndex.from_product([["label"], df_label.columns])
    
    # 7) Concatenate features and label columns, and fill missing values with 0
    df_qlib = pd.concat([df_feature, df_label], axis=1)
    df_qlib = df_qlib.fillna(0)
    
    # 8) Name the index levels and sort the DataFrame
    df_qlib.index.names = ["datetime", "instrument"]
    df_qlib = df_qlib.sort_index(level=["datetime", "instrument"])

    
    # 9) Determine start and end dates from the datetime index level
    start = df_qlib.index.get_level_values("datetime").min()
    end = df_qlib.index.get_level_values("datetime").max()
    
    # 10) Build TSDataSampler (using fillna_type='ffill+bfill' for reindexing)
    sampler = TSDataSampler(df_qlib, start, end, step_len=8, fillna_type='ffill+bfill')
    sampler.data_arr = np.nan_to_num(sampler.data_arr, nan=0.0)
    
    return sampler

# Use the function for training, validation, and test sets:
dl_train = convert_data_qlibformat(df_train)
dl_valid = convert_data_qlibformat(df_valid)
dl_test = convert_data_qlibformat(df_test)


In [191]:

# Save the merged DataFrame tinpuo a pickle file
with open("training_input.pkl", "wb") as f:
    pickle.dump(dl_train, f)
with open("valid_input.pkl", "wb") as f:
    pickle.dump(dl_valid, f)
with open("testing_input.pkl", "wb") as f:
    pickle.dump(dl_test, f)



In [192]:
with open(f'training_input.pkl', 'rb') as f:
    dl_train = pickle.load(f)
with open(f'valid_input.pkl', 'rb') as f:
    dl_valid = pickle.load(f)
with open(f'testing_input.pkl', 'rb') as f:
    dl_test = pickle.load(f)

In [193]:
# After loading:
with open('training_input.pkl', 'rb') as f:
    ts_sampler = pickle.load(f)

print(type(ts_sampler))
print(dir(ts_sampler))
print(ts_sampler.__dict__)

# Extract the data array from the sampler
data_arr = ts_sampler.data_arr

# Convert the data array to a DataFrame
data_df = pd.DataFrame(data_arr)

# Display the DataFrame
print(data_df.head())

# Check for any NaN values in the data array
if np.isnan(data_arr).any():
    print("NaN found in ts_sampler.data_arr")
else:
    print("No NaN found in ts_sampler.data_arr")


<class 'qlib.data.dataset.TSDataSampler'>
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_get_indices', '_get_row_col', 'build_index', 'config', 'data_arr', 'data_index', 'empty', 'end', 'fillna_type', 'flt_idx_map', 'get_index', 'idx_arr', 'idx_df', 'idx_map', 'idx_map2arr', 'nan_idx', 'slice_idx_map_and_data_index', 'start', 'step_len']
{'start': Timestamp('2022-01-03 00:00:00'), 'end': Timestamp('2023-12-12 00:00:00'), 'step_len': 8, 'fillna_type': 'ffill+bfill', 'data_arr': array([[-0.12984978, -0.12291   , -0.12327566, ..., -0.6788242 ,
        -0.56645627,  0.        ],
       [-0.11662426, -0.1227468 , -0.11934439, ...,  0.2

In [194]:
for i, batch in enumerate(dl_valid):
    if i >= 237:
        print(f"Batch {i}:")
        print("  Type:", type(batch))            # Should be <class 'numpy.ndarray'>
        print("  Shape:", batch.shape)           # e.g. (batch_size, step_len, feature_dim)
        # Optionally, print some values:
        print("  Sample data:", batch[:2])       # First 2 samples
    if i >= 241:
        break



Batch 237:
  Type: <class 'numpy.ndarray'>
  Shape: (8, 124)
  Sample data: [[-2.43289149e-01 -2.46306368e-01 -2.43858674e-01 -2.47532560e-01
  -3.98698636e-01 -3.27665307e-01 -2.45911137e-01 -2.47725426e-01
  -1.02644249e+00 -9.84453489e-01  5.75605359e-01 -1.20668605e+00
   6.94164886e-01  3.23435797e-02  3.07358878e-01  5.21301495e-02
   4.03111401e-02 -1.00542278e+00 -8.19620750e-02  4.91861806e-01
   2.70295860e-01 -2.75257523e-01  7.67693536e-01  3.26863839e-01
   1.23769223e+00 -1.05712015e+00 -1.51356829e+00  1.01558536e+00
   6.90035928e-01  5.32010454e-02  2.90472390e-01  4.21161981e-01
  -5.27609484e-01  2.51350197e-01 -1.12505228e-01  2.60720220e-01
  -1.03690879e+00  1.33025863e-01  1.02295898e+00  1.11396153e+00
  -6.86794743e-01  5.64957746e-01 -4.13951819e-01  2.42607067e-01
  -1.78087044e+00  1.87026359e-01  1.35649328e-01 -8.69369121e-02
  -1.04583516e+00  5.09575975e-01 -1.30006143e+00  6.86579188e-01
  -7.55642089e-01 -4.05423174e-02  1.61801831e-01 -5.49048952e-01


In [195]:
d_feat = 9
d_model = 256
t_nhead = 4
s_nhead = 2
dropout = 0.5
gate_input_start_index = 9
gate_input_end_index = 123

beta = 5

n_epoch = 100
lr = 1e-4
GPU = 0
train_stop_loss_thred = 0.001


ic = []
icir = []
ric = []
ricir = []

In [196]:
import numpy as np
import pandas as pd
import copy

from torch.utils.data import DataLoader
from torch.utils.data import Sampler
import torch
import torch.optim as optim

def calc_ic(pred, label):
    df = pd.DataFrame({'pred':pred, 'label':label})
    ic = df['pred'].corr(df['label'])
    ric = df['pred'].corr(df['label'], method='spearman')
    return ic, ric

def zscore(x):
    if x.numel() == 0:
        print(x) 
        print(x.std(unbiased=False))
    return (x - x.mean()).div(x.std())

def drop_extreme(x):
    sorted_tensor, indices = x.sort()
    N = x.shape[0]
    percent_2_5 = int(0.025*N)  
    # Exclude top 2.5% and bottom 2.5% values
    filtered_indices = indices[percent_2_5:-percent_2_5]
    mask = torch.zeros_like(x, device=x.device, dtype=torch.bool)
    mask[filtered_indices] = True
    return mask, x[mask]

def drop_na(x):
    N = x.shape[0]
    mask = ~x.isnan()
    return mask, x[mask]

class DailyBatchSamplerRandom(Sampler):
    def __init__(self, data_source, shuffle=False):
        self.data_source = data_source
        self.shuffle = shuffle
        # calculate number of samples in each batch
        
        self.daily_count = pd.Series(index=self.data_source.get_index()).groupby("datetime").size().values
        self.daily_index = np.roll(np.cumsum(self.daily_count), 1)  # calculate begin index of each batch
        self.daily_index[0] = 0

    def __iter__(self):
        if self.shuffle:
            index = np.arange(len(self.daily_count))
            np.random.shuffle(index)
            for i in index:
                yield np.arange(self.daily_index[i], self.daily_index[i] + self.daily_count[i])
        else:
            for idx, count in zip(self.daily_index, self.daily_count):
                yield np.arange(idx, idx + count)

    def __len__(self):
        return len(self.data_source)


class SequenceModel():
    def __init__(self, n_epochs, lr, GPU=None, seed=None, train_stop_loss_thred=None, save_path = 'model/', save_prefix= ''):
        self.n_epochs = n_epochs
        self.lr = lr
        self.device = torch.device(f"cuda:{GPU}" if torch.cuda.is_available() else "cpu")
        self.seed = seed
        self.train_stop_loss_thred = train_stop_loss_thred

        if self.seed is not None:
            np.random.seed(self.seed)
            torch.manual_seed(self.seed)
            torch.cuda.manual_seed_all(self.seed)
            torch.backends.cudnn.deterministic = True
        self.fitted = -1

        self.model = None
        self.train_optimizer = None

        self.save_path = save_path
        self.save_prefix = save_prefix


    def init_model(self):
        if self.model is None:
            raise ValueError("model has not been initialized")

        self.train_optimizer = optim.Adam(self.model.parameters(), self.lr)
        self.model.to(self.device)

    def loss_fn(self, pred, label):
        mask = ~torch.isnan(label)
        loss = (pred[mask]-label[mask])**2
        return torch.mean(loss)

    def train_epoch(self, data_loader):
        self.model.train()
        losses = []

        for data in data_loader:
            data = torch.squeeze(data, dim=0)
            '''
            data.shape: (N, T, F)
            N - number of stocks
            T - length of lookback_window, 8
            F - 158 factors + 63 market information + 1 label           
            '''
            feature = data[:, :, 0:-1].to(self.device)
            label = data[:, -1, -1].to(self.device)
            
            # Additional process on labels
            # If you use original data to train, you won't need the following lines because we already drop extreme when we dumped the data.
            # If you use the opensource data to train, use the following lines to drop extreme labels.
            #########################
            #mask, label = drop_extreme(label)
            #feature = feature[mask, :, :]
            #label = zscore(label) # CSZscoreNorm
            #########################

            pred = self.model(feature.float())    
            loss = self.loss_fn(pred, label)
            losses.append(loss.item())

            self.train_optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_value_(self.model.parameters(), 3.0)
            self.train_optimizer.step()
        
        return float(np.mean(losses))

    def test_epoch(self, data_loader):
        self.model.eval()
        losses = []

        for data in data_loader:
            data = torch.squeeze(data, dim=0)
            feature = data[:, :, 0:-1].to(self.device)
            label = data[:, -1, -1].to(self.device)

            # You cannot drop extreme labels for test. 
            label = zscore(label)
                        
            pred = self.model(feature.float())
            loss = self.loss_fn(pred, label)
            losses.append(loss.item())

        return float(np.mean(losses))

    def _init_data_loader(self, data, shuffle=True, drop_last=True):
        sampler = DailyBatchSamplerRandom(data, shuffle)
        data_loader = DataLoader(data, sampler=sampler, drop_last=drop_last)
        return data_loader

    def load_param(self, param_path):
        self.model.load_state_dict(torch.load(param_path, map_location=self.device))
        self.fitted = 'Previously trained.'

    def fit(self, dl_train, dl_valid=None):
        train_loader = self._init_data_loader(dl_train, shuffle=True, drop_last=True)
        best_param = None
        for step in range(self.n_epochs):
            train_loss = self.train_epoch(train_loader)
            self.fitted = step
            if dl_valid:
                predictions, metrics = self.predict(dl_valid)
                print("Epoch %d, train_loss %.6f, valid ic %.4f, icir %.3f, rankic %.4f, rankicir %.3f." % (step, train_loss, metrics['IC'],  metrics['ICIR'],  metrics['RIC'],  metrics['RICIR']))
            else: print("Epoch %d, train_loss %.6f" % (step, train_loss))
        
            if train_loss <= self.train_stop_loss_thred:
                best_param = copy.deepcopy(self.model.state_dict())
                torch.save(best_param, f'{self.save_path}/{self.save_prefix}_{self.seed}.pkl')
                break
        

        

    def predict(self, dl_test):
        # string cannot be compared with int,add one more condition here
        if isinstance(self.fitted,(int, float)) and self.fitted<0:
        #if self.fitted<0:
            raise ValueError("model is not fitted yet!")
        else:
            print('Epoch:', self.fitted)

        test_loader = self._init_data_loader(dl_test, shuffle=False, drop_last=False)

        preds = []
        ic = []
        ric = []

        self.model.eval()
        for data in test_loader:
            data = torch.squeeze(data, dim=0)
            feature = data[:, :, 0:-1].to(self.device)
            label = data[:, -1, -1]
            
            # nan label will be automatically ignored when compute metrics.
            # zscorenorm will not affect the results of ranking-based metrics.

            with torch.no_grad():
                pred = self.model(feature.float()).detach().cpu().numpy()
            preds.append(pred.ravel())

            daily_ic, daily_ric = calc_ic(pred, label.detach().numpy())
            ic.append(daily_ic)
            ric.append(daily_ric)

        predictions = pd.Series(np.concatenate(preds), index=dl_test.get_index())

        metrics = {
            'IC': np.mean(ic),
            'ICIR': np.mean(ic)/np.std(ic),
            'RIC': np.mean(ric),
            'RICIR': np.mean(ric)/np.std(ric)
        }

        return predictions, metrics


In [186]:
import torch
from torch import nn
from torch.nn.modules.linear import Linear
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.normalization import LayerNorm
import math



class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=100):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

    def forward(self, x):
        return x + self.pe[:x.shape[1], :]


class SAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout):
        super().__init__()

        self.d_model = d_model
        self.nhead = nhead
        self.temperature = math.sqrt(self.d_model/nhead)

        self.qtrans = nn.Linear(d_model, d_model, bias=False)
        self.ktrans = nn.Linear(d_model, d_model, bias=False)
        self.vtrans = nn.Linear(d_model, d_model, bias=False)

        attn_dropout_layer = []
        for i in range(nhead):
            attn_dropout_layer.append(Dropout(p=dropout))
        self.attn_dropout = nn.ModuleList(attn_dropout_layer)

        # input LayerNorm
        self.norm1 = LayerNorm(d_model, eps=1e-5)

        # FFN layerNorm
        self.norm2 = LayerNorm(d_model, eps=1e-5)
        self.ffn = nn.Sequential(
            Linear(d_model, d_model),
            nn.ReLU(),
            Dropout(p=dropout),
            Linear(d_model, d_model),
            Dropout(p=dropout)
        )

    def forward(self, x):
        x = self.norm1(x)
        q = self.qtrans(x).transpose(0,1)
        k = self.ktrans(x).transpose(0,1)
        v = self.vtrans(x).transpose(0,1)

        dim = int(self.d_model/self.nhead)
        att_output = []
        for i in range(self.nhead):
            if i==self.nhead-1:
                qh = q[:, :, i * dim:]
                kh = k[:, :, i * dim:]
                vh = v[:, :, i * dim:]
            else:
                qh = q[:, :, i * dim:(i + 1) * dim]
                kh = k[:, :, i * dim:(i + 1) * dim]
                vh = v[:, :, i * dim:(i + 1) * dim]

            atten_ave_matrixh = torch.softmax(torch.matmul(qh, kh.transpose(1, 2)) / self.temperature, dim=-1)
            if self.attn_dropout:
                atten_ave_matrixh = self.attn_dropout[i](atten_ave_matrixh)
            att_output.append(torch.matmul(atten_ave_matrixh, vh).transpose(0, 1))
        att_output = torch.concat(att_output, dim=-1)

        # FFN
        xt = x + att_output
        xt = self.norm2(xt)
        att_output = xt + self.ffn(xt)

        return att_output


class TAttention(nn.Module):
    def __init__(self, d_model, nhead, dropout):
        super().__init__()
        self.d_model = d_model
        self.nhead = nhead
        self.qtrans = nn.Linear(d_model, d_model, bias=False)
        self.ktrans = nn.Linear(d_model, d_model, bias=False)
        self.vtrans = nn.Linear(d_model, d_model, bias=False)

        self.attn_dropout = []
        if dropout > 0:
            for i in range(nhead):
                self.attn_dropout.append(Dropout(p=dropout))
            self.attn_dropout = nn.ModuleList(self.attn_dropout)

        # input LayerNorm
        self.norm1 = LayerNorm(d_model, eps=1e-5)
        # FFN layerNorm
        self.norm2 = LayerNorm(d_model, eps=1e-5)
        # FFN
        self.ffn = nn.Sequential(
            Linear(d_model, d_model),
            nn.ReLU(),
            Dropout(p=dropout),
            Linear(d_model, d_model),
            Dropout(p=dropout)
        )

    def forward(self, x):
        x = self.norm1(x)
        q = self.qtrans(x)
        k = self.ktrans(x)
        v = self.vtrans(x)

        dim = int(self.d_model / self.nhead)
        att_output = []
        for i in range(self.nhead):
            if i==self.nhead-1:
                qh = q[:, :, i * dim:]
                kh = k[:, :, i * dim:]
                vh = v[:, :, i * dim:]
            else:
                qh = q[:, :, i * dim:(i + 1) * dim]
                kh = k[:, :, i * dim:(i + 1) * dim]
                vh = v[:, :, i * dim:(i + 1) * dim]
            atten_ave_matrixh = torch.softmax(torch.matmul(qh, kh.transpose(1, 2)), dim=-1)
            if self.attn_dropout:
                atten_ave_matrixh = self.attn_dropout[i](atten_ave_matrixh)
            att_output.append(torch.matmul(atten_ave_matrixh, vh))
        att_output = torch.concat(att_output, dim=-1)

        # FFN
        xt = x + att_output
        xt = self.norm2(xt)
        att_output = xt + self.ffn(xt)

        return att_output


class Gate(nn.Module):
    def __init__(self, d_input, d_output,  beta=1.0):
        super().__init__()
        self.trans = nn.Linear(d_input, d_output)
        self.d_output =d_output
        self.t = beta

    def forward(self, gate_input):
        output = self.trans(gate_input)
        output = torch.softmax(output/self.t, dim=-1)
        return self.d_output*output


class TemporalAttention(nn.Module):
    def __init__(self, d_model):
        super().__init__()
        self.trans = nn.Linear(d_model, d_model, bias=False)

    def forward(self, z):
        h = self.trans(z) # [N, T, D]
        query = h[:, -1, :].unsqueeze(-1)
        lam = torch.matmul(h, query).squeeze(-1)  # [N, T, D] --> [N, T]
        lam = torch.softmax(lam, dim=1).unsqueeze(1)
        output = torch.matmul(lam, z).squeeze(1)  # [N, 1, T], [N, T, D] --> [N, 1, D]
        return output


class MASTER(nn.Module):
    def __init__(self, d_feat, d_model, t_nhead, s_nhead, T_dropout_rate, S_dropout_rate, gate_input_start_index, gate_input_end_index, beta):
        super(MASTER, self).__init__()
        # market
        self.gate_input_start_index = gate_input_start_index
        self.gate_input_end_index = gate_input_end_index
        self.d_gate_input = (gate_input_end_index - gate_input_start_index) # F'
        self.feature_gate = Gate(self.d_gate_input, d_feat, beta=beta)

        self.layers = nn.Sequential(
            # feature layer
            nn.Linear(d_feat, d_model),
            PositionalEncoding(d_model),
            # intra-stock aggregation
            TAttention(d_model=d_model, nhead=t_nhead, dropout=T_dropout_rate),
            # inter-stock aggregation
            SAttention(d_model=d_model, nhead=s_nhead, dropout=S_dropout_rate),
            TemporalAttention(d_model=d_model),
            # decoder
            nn.Linear(d_model, 1)
        )

    def forward(self, x):
        src = x[:, :, :self.gate_input_start_index] # N, T, D
        gate_input = x[:, -1, self.gate_input_start_index:self.gate_input_end_index]
        src = src * torch.unsqueeze(self.feature_gate(gate_input), dim=1)
       
        output = self.layers(src).squeeze(-1)

        return output


class MASTERModel(SequenceModel):
    def __init__(
            self, d_feat, d_model, t_nhead, s_nhead, gate_input_start_index, gate_input_end_index,
            T_dropout_rate, S_dropout_rate, beta, **kwargs,
    ):
        super(MASTERModel, self).__init__(**kwargs)
        self.d_model = d_model
        self.d_feat = d_feat

        self.gate_input_start_index = gate_input_start_index
        self.gate_input_end_index = gate_input_end_index

        self.T_dropout_rate = T_dropout_rate
        self.S_dropout_rate = S_dropout_rate
        self.t_nhead = t_nhead
        self.s_nhead = s_nhead
        self.beta = beta

        self.init_model()

    def init_model(self):
        self.model = MASTER(d_feat=self.d_feat, d_model=self.d_model, t_nhead=self.t_nhead, s_nhead=self.s_nhead,
                                   T_dropout_rate=self.T_dropout_rate, S_dropout_rate=self.S_dropout_rate,
                                   gate_input_start_index=self.gate_input_start_index,
                                   gate_input_end_index=self.gate_input_end_index, beta=self.beta)
        super(MASTERModel, self).init_model()


In [187]:
#from master import MASTERModel

for seed in [0, 1, 2, 3, 4]:
    model = MASTERModel(
        d_feat = d_feat, d_model = d_model, t_nhead = t_nhead, s_nhead = s_nhead, T_dropout_rate=dropout, S_dropout_rate=dropout,
        beta=beta, gate_input_end_index=gate_input_end_index, gate_input_start_index=gate_input_start_index,
        n_epochs=n_epoch, lr = lr, GPU = GPU, seed = seed, train_stop_loss_thred = train_stop_loss_thred,
        save_path='model', save_prefix=f'model_training'
    )

    start = time.time()
    # Train
    model.fit(dl_train, dl_valid)

    print("Model Trained.")

    # Test
    predictions, metrics = model.predict(dl_test)
    
    running_time = time.time()-start

    print('Seed: {:d} time cost : {:.2f} sec'.format(seed, running_time))
    print(metrics)

    ic.append(metrics['IC'])
    icir.append(metrics['ICIR'])
    ric.append(metrics['RIC'])
    ricir.append(metrics['RICIR'])

  

Epoch: 0
Epoch 0, train_loss 0.031807, valid ic 0.4159, icir 1.664, rankic 0.4438, rankicir 1.930.
Epoch: 1
Epoch 1, train_loss 0.004458, valid ic 0.5351, icir 2.523, rankic 0.5842, rankicir 3.015.
Epoch: 2
Epoch 2, train_loss 0.003333, valid ic 0.5956, icir 3.466, rankic 0.6165, rankicir 3.724.
Epoch: 3
Epoch 3, train_loss 0.001796, valid ic 0.5292, icir 3.301, rankic 0.5695, rankicir 3.909.
Epoch: 4
Epoch 4, train_loss 0.005339, valid ic 0.5801, icir 3.427, rankic 0.5982, rankicir 3.655.
Epoch: 5
Epoch 5, train_loss 0.001014, valid ic 0.5821, icir 2.943, rankic 0.6182, rankicir 3.152.
Epoch: 6
Epoch 6, train_loss 0.001245, valid ic 0.5607, icir 2.756, rankic 0.5900, rankicir 2.849.
Epoch: 7
Epoch 7, train_loss 0.001257, valid ic 0.5791, icir 2.648, rankic 0.6002, rankicir 2.543.
Epoch: 8
Epoch 8, train_loss 0.001393, valid ic 0.6030, icir 2.948, rankic 0.6108, rankicir 2.806.
Epoch: 9
Epoch 9, train_loss 0.001566, valid ic 0.6781, icir 4.594, rankic 0.6863, rankicir 4.120.
Epoch: 10


In [188]:
# Load and Test
######################################################################################
for seed in [0]:
    param_path = f'model/model_training_{seed}.pkl'

    print(f'Model Loaded from {param_path}')
    model = MASTERModel(
            d_feat = d_feat, d_model = d_model, t_nhead = t_nhead, s_nhead = s_nhead, T_dropout_rate=dropout, S_dropout_rate=dropout,
            beta=beta, gate_input_end_index=gate_input_end_index, gate_input_start_index=gate_input_start_index,
            n_epochs=n_epoch, lr = lr, GPU = GPU, seed = seed, train_stop_loss_thred = train_stop_loss_thred,
            save_path='model', save_prefix=f'model_prediction'
        )
    model.load_param(param_path)
    predictions, metrics = model.predict(dl_test)
    print(metrics)

    ic.append(metrics['IC'])
    icir.append(metrics['ICIR'])
    ric.append(metrics['RIC'])
    ricir.append(metrics['RICIR'])
    
######################################################################################

print("IC: {:.4f} pm {:.4f}".format(np.mean(ic), np.std(ic)))
print("ICIR: {:.4f} pm {:.4f}".format(np.mean(icir), np.std(icir)))
print("RIC: {:.4f} pm {:.4f}".format(np.mean(ric), np.std(ric)))
print("RICIR: {:.4f} pm {:.4f}".format(np.mean(ricir), np.std(ricir)))

Model Loaded from model/model_training_0.pkl
Epoch: Previously trained.
{'IC': np.float64(0.744734238730873), 'ICIR': np.float64(8.485826422776752), 'RIC': np.float64(0.7600683090616813), 'RICIR': np.float64(7.313945772590813)}
IC: 0.7081 pm 0.0513
ICIR: 6.2046 pm 1.8534
RIC: 0.7359 pm 0.0444
RICIR: 5.6843 pm 1.4672
